<a href="https://colab.research.google.com/github/newstage75/ML_study/blob/master/%E6%9C%80%E7%9F%ADPytorch/%EF%BC%95%E7%AB%A0%E7%B7%9A%E5%BD%A2%E5%9B%9E%E5%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#5章線形回帰

In [5]:
# 必要ライブラリの導入

!pip install japanize_matplotlib | tail -n 1
!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1

In [6]:
# 必要ライブラリのインポート

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

In [7]:
# PyTorch関連ライブラリ
import torch
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot

In [9]:
# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=4)

In [10]:
#5.3線形関数（nn.Linear）
torch.manual_seed(123)

l1 = nn.Linear(1, 1)

print(l1)

Linear(in_features=1, out_features=1, bias=True)


In [11]:
#named_parametersという関数を呼びだすと、（パラメータ名、パラメータ参照）のリストを返す
#パラメータ名、パラメータ値、shapeの表示

for param in l1.named_parameters():
  print('name: ', param[0])
  print('tensor: ', param[1])
  print('shape: ', param[1].shape)

name:  weight
tensor:  Parameter containing:
tensor([[-0.4078]], requires_grad=True)
shape:  torch.Size([1, 1])
name:  bias
tensor:  Parameter containing:
tensor([0.0331], requires_grad=True)
shape:  torch.Size([1])


In [12]:
#初期値設定
nn.init.constant_(l1.weight, 2.0)
nn.init.constant_(l1.bias, 1.0)

#結果確認
print(l1.weight)
print(l1.bias)

Parameter containing:
tensor([[2.]], requires_grad=True)
Parameter containing:
tensor([1.], requires_grad=True)


In [13]:
#再度パラメータ表示の確認テスト
for param in l1.named_parameters():
  print('name: ', param[0])
  print('tensor: ', param[1])
  print('shape: ', param[1].shape)

name:  weight
tensor:  Parameter containing:
tensor([[2.]], requires_grad=True)
shape:  torch.Size([1, 1])
name:  bias
tensor:  Parameter containing:
tensor([1.], requires_grad=True)
shape:  torch.Size([1])


In [14]:
#テスト用ダミーデータをこの関数にかけて１次関数として動作していることを確認
x_np = np.arange(-2, 2.1, 1)
x = torch.tensor(x_np).float()
#サイズを(N,1)に変換
x = x.view(-1, 1)

print(x.shape)
print(x)

torch.Size([5, 1])
tensor([[-2.],
        [-1.],
        [ 0.],
        [ 1.],
        [ 2.]])


In [15]:
y = l1(x)
print(y.shape)
print(y.data)

torch.Size([5, 1])
tensor([[-3.],
        [-1.],
        [ 1.],
        [ 3.],
        [ 5.]])
